To do:
- group the original dataframe (candidates only) to get each candidate's info and the number of emails sent (email id)
- for house candidates, make sure each candidate is labeled with their correct opponent by position
- for senate candidates, find top two candidates per state & make sure candidates with black opponents are labeled correctly
- merge the og dataframe with both house & senate by candidate last name & state

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)

# House Data

In [2]:
house = pd.read_csv('house.csv', usecols=['year', 'state', 'district', 'candidate', 'party', 'candidatevotes'])
house = house.loc[house['year'] == 2020]
house['state'] = house['state'].str.title()
house['candidate'] = house['candidate'].str.title()

In [3]:
house = house.loc[(house['party'] == 'DEMOCRAT') | (house['party'] == 'REPUBLICAN')]

In [4]:
grouped_house = house.groupby(['state', 'district'])

def top_two(group):
    top_two_candidates = group.nlargest(2, 'candidatevotes', keep='first')
    return top_two_candidates

general_house_candidates = []

for group_name, group_data in grouped_house:
    general_house_candidates.append(top_two(group_data))
    
house = pd.concat(general_house_candidates).reset_index(drop=True)

In [5]:
def last_name(name):
    name = name.strip().split(" ")
    return name[-1]

house['last_name'] = house['candidate'].apply(last_name)

In [6]:
def find_opponent(candidate):
    candidate_row = house.loc[house['candidate'] == candidate]
    state = candidate_row['state'].values[0]
    dist = candidate_row['district'].values[0]
    dist_candidates = house.loc[(house['state'] == state) & (house['district'] == dist)]['candidate'].to_list()
    dist_candidates.remove(candidate)
    if dist_candidates:
        return dist_candidates[0]
    return 'none'

house['opponent'] = house['candidate'].apply(find_opponent)
house['opponent_last_name'] = house['opponent'].apply(last_name)

In [7]:
house

,year,state,district,candidate,party,candidatevotes,last_name,opponent,opponent_last_name
0,2020,Alabama,1,Jerry Carl,REPUBLICAN,211825,Carl,James Averhart,Averhart
1,2020,Alabama,1,James Averhart,DEMOCRAT,116949,Averhart,Jerry Carl,Carl
2,2020,Alabama,2,Barry Moore,REPUBLICAN,197996,Moore,Phyllis Harvey-Hall,Harvey-Hall
3,2020,Alabama,2,Phyllis Harvey-Hall,DEMOCRAT,105286,Harvey-Hall,Barry Moore,Moore
4,2020,Alabama,3,Mike Rogers,REPUBLICAN,217384,Rogers,Adia Winfrey,Winfrey
5,2020,Alabama,3,Adia Winfrey,DEMOCRAT,104595,Winfrey,Mike Rogers,Rogers
6,2020,Alabama,4,Robert B Aderholt,REPUBLICAN,261553,Aderholt,Rick Neighbors,Neighbors
7,2020,Alabama,4,Rick Neighbors,DEMOCRAT,56237,Neighbors,Robert B Aderholt,Aderholt
8,2020,Alabama,5,Mo Brooks,REPUBLICAN,253094,Brooks,none,none
9,2020,Alabama,6,Gary J Palmer,REPUBLICAN,274160,Palmer,none,none


# Senate data

In [8]:
senate = pd.read_csv('senate.csv', usecols=['state', 'candidate', 'candidatevotes', 'party_simplified'])
senate = senate.loc[(senate['party_simplified'] == 'DEMOCRAT') | (senate['party_simplified'] == 'REPUBLICAN')]
senate['state'] = senate['state'].str.title()
senate['candidate'] = senate['candidate'].str.title()
senate = senate.replace('Mark C. Curran Jr.', 'Mark Curran')

In [9]:
grouped_senate = senate.groupby('state')

general_senate_candidates = []

for group_name, group_data in grouped_senate:
    general_senate_candidates.append(top_two(group_data))
    
senate = pd.concat(general_senate_candidates).reset_index(drop=True)

senate['last_name'] = senate['candidate'].apply(last_name)

def senate_opponent(candidate):
    candidate_row = senate.loc[senate['candidate'] == candidate]
    state = candidate_row['state'].values[0]
    state_candidates = senate.loc[senate['state'] == state]['candidate'].to_list()
    state_candidates.remove(candidate)
    if state_candidates:
        return state_candidates[0]
    return 'none'

senate['opponent'] = senate['candidate'].apply(senate_opponent)
senate['opponent_last_name'] = senate['opponent'].apply(last_name)

warnock_ga = {'state': 'Georgia', 'candidate': 'Raphael Warnock', 'candidatevotes': 0, 'party_simplified': 'DEMOCRAT', 'last_name': 'Warnock', 'opponent': 'Kelly Loeffler', 'opponent_last_name': 'Loeffler'}
loeffler_ga = {'state': 'Georgia', 'candidate': 'Kelly Loeffler', 'candidatevotes': 0, 'party_simplified': 'REPUBLICAN', 'last_name': 'Loeffler', 'opponent': 'Raphael Warnock', 'opponent_last_name': 'Warnock'}

senate.loc[67] = warnock_ga
senate.loc[68] = loeffler_ga

In [10]:
senate

,state,candidate,candidatevotes,party_simplified,last_name,opponent,opponent_last_name
0,Alabama,Tommy Tuberville,1392076,REPUBLICAN,Tuberville,Doug Jones,Jones
1,Alabama,Doug Jones,920478,DEMOCRAT,Jones,Tommy Tuberville,Tuberville
2,Alaska,Dan Sullivan,191112,REPUBLICAN,Sullivan,Al Gross,Gross
3,Alaska,Al Gross,146068,DEMOCRAT,Gross,Dan Sullivan,Sullivan
4,Arizona,Mark Kelly,1716467,DEMOCRAT,Kelly,Martha Mcsally,Mcsally
5,Arizona,Martha Mcsally,1637661,REPUBLICAN,Mcsally,Mark Kelly,Kelly
6,Arkansas,Tom Cotton,793871,REPUBLICAN,Cotton,none,none
7,Colorado,John W. Hickenlooper,1731114,DEMOCRAT,Hickenlooper,Cory Gardner,Gardner
8,Colorado,Cory Gardner,1429492,REPUBLICAN,Gardner,John W. Hickenlooper,Hickenlooper
9,Delaware,Christopher A. Coons,291804,DEMOCRAT,Coons,Lauren Witzke,Witzke


# Corpus sender data

In [11]:
df = pd.read_csv('../corpus_sample/primary_candidates_corpus.csv', usecols=['name', 'office_sought', 'party_affiliation', 'office_level', 'body_text', 'district_type', 'uid_email'])
candidates = df.loc[((df['district_type'] == 'State') | (df['district_type'] == 'Congress'))].groupby(['name', 'office_sought', 'party_affiliation', 'office_level', 'district_type']).count()

In [12]:
threshold = 10
filtered_by_count = candidates.loc[candidates['uid_email'] >= threshold]
candidates = filtered_by_count.rename(columns={'uid_email': 'total_emails_sent'}, index={'Adair Ford Boroughs': 'Adair Boroughs', 'Anthony G. Brown': 'Anthony Brown', 'Carolyn B. Maloney': 'Carolyn Maloney', 'Chris Bubser': 'Christine Bubser', 'Christopher Jacobs': 'Chris Jacobs', 'Linda Sánchez': 'Linda Sanchez', })
candidates.to_csv('grouped_candidate_info.csv')

In [13]:
candidates = pd.read_csv('grouped_candidate_info.csv', index_col=0).reset_index()

In [14]:
def get_state(office):
    split_state = office.strip().split(" ")
    if split_state[1] == "House":
        return " ".join(split_state[2:-2])
    elif split_state[1] == "Senate":
        return " ".join(split_state[2:])
    
candidates['last_name'] = candidates['name'].apply(last_name)
candidates['state'] = candidates['office_sought'].apply(get_state)

In [15]:
candidates

,name,office_sought,party_affiliation,office_level,district_type,body_text,total_emails_sent,last_name,state
0,Aaron Godfrey,U.S. House Ohio District 16,Democratic Party,Federal,Congress,42,42,Godfrey,Ohio
1,Aaron Swisher,U.S. House Idaho District 2,Democratic Party,Federal,Congress,17,17,Swisher,Idaho
2,Abby Broyles,U.S. Senate Oklahoma,Democratic Party,Federal,State,306,306,Broyles,Oklahoma
3,Abigail Spanberger,U.S. House Virginia District 7,Democratic Party,Federal,Congress,478,478,Spanberger,Virginia
4,Adair Boroughs,U.S. House South Carolina District 2,Democratic Party,Federal,Congress,1090,1090,Boroughs,South Carolina
5,Adam Bolanos Scow,U.S. House California District 20,Democratic Party,Federal,Congress,22,22,Scow,California
6,Adam Christensen,U.S. House Florida District 3,Democratic Party,Federal,Congress,41,41,Christensen,Florida
7,Adam Hattersley,U.S. House Florida District 15,Democratic Party,Federal,Congress,118,137,Hattersley,Florida
8,Adam Schiff,U.S. House California District 28,Democratic Party,Federal,Congress,38,38,Schiff,California
9,Adrienne Bell,U.S. House Texas District 14,Democratic Party,Federal,Congress,99,99,Bell,Texas


In [16]:
house_merge = candidates.merge(house, on=['state', 'last_name'], how='outer')
house_merge = house_merge.loc[~house_merge['name'].isna()]
senate_merge = house_merge.merge(senate, on=['state', 'last_name'], how='outer')
merged = senate_merge.drop(columns=['year', 'district', 'candidate_x', 'party', 'candidatevotes_x', 'candidate_y', 'candidatevotes_y', 'party_simplified'])
merged['opponent_x'] = merged.apply(
    lambda row: row['opponent_x'] if pd.notnull(row['opponent_x']) else row['opponent_y'],
    axis=1)
merged['opponent_last_name_x'] = merged.apply(
    lambda row: row['opponent_last_name_x'] if pd.notnull(row['opponent_last_name_x']) else row['opponent_last_name_y'],
    axis=1)
merged = merged.drop(columns=['opponent_y', 'opponent_last_name_y']).rename(columns={'opponent_x': 'opponent', 'opponent_last_name_x': 'opponent_last_name'})
merged = merged.loc[~merged['opponent'].isna()]

In [17]:
black_candidates = pd.read_csv('black_congress_candidates_modified.csv', index_col=0)
black_candidates = black_candidates.rename(columns={'last_name': 'opponent_last_name', 'office_sought': 'position'})
black_candidates['is_black'] = True

In [18]:
merged = merged.merge(black_candidates, on=['state', 'opponent_last_name'], how='left')

In [19]:
merged = merged.drop(columns=['candidate_name', 'party', 'position', '%_votes'])
merged['is_black'].fillna(False, inplace=True)

In [20]:
filtered_candidates = merged['name'].to_list()

In [21]:
general_candidates_corpus = df.loc[df['name'].isin(filtered_candidates)]

In [22]:
general_candidates_corpus.to_csv('../corpus_sample/general_candidates_corpus.csv')

In [23]:
duplicates = {'Young Kim': 'Jimmy Gomez',
              'Carolyn Maloney': 'Chele Farley',
              'Sean Maloney': 'Carlos Santiago-Cano',
              'Hank Johnson': 'Rick Allen',
              'Jimmy Gomez': 'Sara Jacobs',
              'Georgette Gomez': 'David Kim',
              'Christy Smith': 'Mark Takano'}

for candidate_name in duplicates:
    drop_index = merged.loc[(merged['name'] == candidate_name) & (merged['opponent'] == duplicates[candidate_name])].index
    merged = merged.drop(drop_index)

In [24]:
merged.loc[merged['name'] == 'Georgette Gomez']

,name,office_sought,party_affiliation,office_level,district_type,body_text,total_emails_sent,last_name,state,opponent,opponent_last_name,is_black
134,Georgette Gomez,U.S. House California District 53,Democratic Party,Federal,Congress,170.0,170.0,Gomez,California,Sara Jacobs,Jacobs,False


In [25]:
merged.at[228, 'is_black'] = False
merged.loc[merged['name'] == 'Lou Correa']

,name,office_sought,party_affiliation,office_level,district_type,body_text,total_emails_sent,last_name,state,opponent,opponent_last_name,is_black
228,Lou Correa,U.S. House California District 46,Democratic Party,Federal,Congress,18.0,18.0,Correa,California,James S Waters,Waters,False


In [26]:
merged.to_csv('aggregated_gen_candidates.csv')